In [55]:
import torch
from torch.utils.data import TensorDataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

import pandas as pd
import numpy as np

from tqdm.notebook import tqdm
import re
tqdm.pandas()

In [56]:
df = pd.read_csv("data/rusentitweet_full.csv")
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df.head()

,text,label,id
0,@varlamov @McFaul На,skip,1327934765807308801
1,велл они всё равно что мусор так что ничего с...,negative,1252943181387350017
2,"""трезвая жизнь какая-то такая стрёмная""\r\n(с)...",negative,1323610669061677056
3,Ой какие неожиданные результаты 🤭 https://t.co...,neutral,1336231661160247297
4,@Shvonder_chief @dimsmirnov175 На заборе тоже ...,neutral,1292421736454127617


In [57]:
max_value = len(df["label"].unique())
max_value

5

In [58]:
df["label"].unique()

array(['skip', 'negative', 'neutral', 'speech', 'positive'], dtype=object)

In [59]:
def change_label_to_int(label):
    match label:
        case 'skip':
            label = label.replace('skip', '0')
        case 'negative':
            label = label.replace('negative', '1')
        case 'neutral':
            label = label.replace('neutral', '2')
        case 'speech':
            label = label.replace('speech', '3')
        case 'positive':
            label = label.replace('positive', '4')
    return label

df['int_labels'] = [change_label_to_int(t) for t in df['label']]
df['int_labels'] = df['int_labels'].astype(int)
df.head()

,text,label,id,int_labels
0,@varlamov @McFaul На,skip,1327934765807308801,0
1,велл они всё равно что мусор так что ничего с...,negative,1252943181387350017,1
2,"""трезвая жизнь какая-то такая стрёмная""\r\n(с)...",negative,1323610669061677056,1
3,Ой какие неожиданные результаты 🤭 https://t.co...,neutral,1336231661160247297,2
4,@Shvonder_chief @dimsmirnov175 На заборе тоже ...,neutral,1292421736454127617,2


In [60]:
def process_text(text):
    text = text.replace('\n', '')
    text = text.replace('\r', '')
    text = text.lower()
    text = text.replace('ё', "е")
    text = re.sub('@[^\s]+','tagged', text)
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','link', text)
    text = re.sub(r"[^a-zA-Z0-9А-Яа-я ]","", text)
    return text

df['processed_text'] = [process_text(t) for t in df['text']]
df.head()


,text,label,id,int_labels,processed_text
0,@varlamov @McFaul На,skip,1327934765807308801,0,tagged tagged на
1,велл они всё равно что мусор так что ничего с...,negative,1252943181387350017,1,велл они все равно что мусор так что ничего с...
2,"""трезвая жизнь какая-то такая стрёмная""\r\n(с)...",negative,1323610669061677056,1,трезвая жизнь какаято такая стремнаяс артем az...
3,Ой какие неожиданные результаты 🤭 https://t.co...,neutral,1336231661160247297,2,ой какие неожиданные результаты link
4,@Shvonder_chief @dimsmirnov175 На заборе тоже ...,neutral,1292421736454127617,2,tagged tagged на заборе тоже написаноа там дру...


In [61]:
df = df.drop('id', axis=1)
df = df.drop('text', axis=1)
df = df.drop('label', axis=1)
df

,int_labels,processed_text
0,0,tagged tagged на
1,1,велл они все равно что мусор так что ничего с...
2,1,трезвая жизнь какаято такая стремнаяс артем az...
3,2,ой какие неожиданные результаты link
4,2,tagged tagged на заборе тоже написаноа там дру...
...,...,...
13387,1,все пора спать пиздец словила шизу
13388,2,такими темпами я создам новую секту или органи...
13389,2,ты смотрела аниме завернувшись в одеяло пока м...
13390,1,tagged пиздануться


In [63]:
tokenizer = BertTokenizer.from_pretrained(
    "DeepPavlov/rubert-base-cased",
    do_lower_case = True
)

In [73]:
def tokenize(text):
    res = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=32,
        pad_to_max_length=True,
        return_attention_mask=True,
        return_tensors="pt"
    )
    return pd.Series([res["input_ids"], res["attention_mask"]])

df[["input_ids", "attention_mask"]] = df["processed_text"].progress_apply(tokenize)


  0%|          | 0/13392 [00:00<?, ?it/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/home/l3xxxa/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2346: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [74]:
test_size = 0.3
batch_size = 16
train_df, test_df = train_test_split(
    df,
    test_size=test_size,
    shuffle=True,
    stratify=df["int_labels"].values
)

In [76]:
train_set = TensorDataset(torch.cat(list(train_df["input_ids"].values), dim=0),
                          torch.cat(list(train_df["attention_mask"].values), dim=0), 
                          torch.tensor(train_df["int_labels"].values))
train_dataloader = DataLoader(
            train_set,
            batch_size=batch_size
        )

In [78]:
test_set = TensorDataset(torch.cat(list(test_df["input_ids"].values), dim=0),
                         torch.cat(list(test_df["attention_mask"].values), dim=0), 
                         torch.tensor(test_df["int_labels"].values))
test_dataloader = DataLoader(
            test_set,
            batch_size=batch_size
        )

In [79]:
device = torch.device("cpu")

In [80]:
model = BertForSequenceClassification.from_pretrained(
    "DeepPavlov/rubert-base-cased",
    num_labels = max_value,
    output_attentions = False,
    output_hidden_states = False,
)

model.to(device)

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were n

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

#### Как я понял, мы должны взять модель и функцию обучения из примера. на 5 эпохах, модель будет обучаться где-то около 4 часов (1 эпоха в среднем тратит около 50 минут). При этом это в идеальных условиях, с учетом, что неожиданно не закончится свап, и ось не прибьет процесс, который так нагло начал жрать ресурсы

In [81]:
epochs = 5

optimizer = torch.optim.AdamW(
    model.parameters(), 
    lr = 5e-6,
    eps = 1e-08
)

for _ in tqdm(range(epochs), desc='Epoch'):
    model.train()
    
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0

    for step, batch in tqdm(enumerate(train_dataloader), total=len(train_dataloader)):
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch

        optimizer.zero_grad()

        train_output = model(b_input_ids, 
                             token_type_ids = None, 
                             attention_mask = b_input_mask, 
                             labels = b_labels)
        
        train_output.loss.backward()
        optimizer.step()

        tr_loss += train_output.loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1

    model.eval()

    val_f1 = []

    for batch in tqdm(test_dataloader, total=len(test_dataloader)):
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch

        with torch.no_grad():
          eval_output = model(b_input_ids, 
                              token_type_ids = None, 
                              attention_mask = b_input_mask)
          
        logits = eval_output.logits
        y_pred = torch.argmax(logits, dim = -1)
        
        y_pred = y_pred.detach().cpu().numpy()
        y_true = b_labels.to('cpu').numpy()

        val_f1_value = f1_score(y_true, y_pred, average='macro')
        val_f1.append(val_f1_value)

    print('\n\t - Train loss: {:.4f}'.format(tr_loss / nb_tr_steps))
    print('\t - Validation F1-score: {:.4f}'.format(sum(val_f1)/len(val_f1)))

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/586 [00:00<?, ?it/s]